In [1]:
from synthetic_pages.datasets.synthetic_datamodule_cubes import SyntheticInstanceCubesDataset

In [ ]:
dataset = SyntheticInstanceCubesDataset(
    "../vast-scrolls/reference_volume.nrrd", 
    "../vast-scrolls/reference_labels.nrrd")

: 

: 

: 

In [ ]:
batch = next(iter(dataset))


: 

: 